In [6]:
import pandas as pd
import numpy as np
import pm4py
from pm4py.objects.log.util import sorting
from scipy.stats import wasserstein_distance
from sklearn import preprocessing

from log_distance_measures.config import EventLogIDs, AbsoluteTimestampType, discretize_to_hour
from log_distance_measures.control_flow_log_distance import control_flow_log_distance
from log_distance_measures.n_gram_distribution import n_gram_distribution_distance
from log_distance_measures.absolute_event_distribution import absolute_event_distribution_distance
from log_distance_measures.case_arrival_distribution import case_arrival_distribution_distance
from log_distance_measures.circadian_event_distribution import circadian_event_distribution_distance
from log_distance_measures.relative_event_distribution import relative_event_distribution_distance
from log_distance_measures.work_in_progress import work_in_progress_distance
from log_distance_measures.cycle_time_distribution import cycle_time_distribution_distance

import warnings
warnings.filterwarnings("ignore")

In [7]:
import re
def extract_first_float(cell):
    if isinstance(cell, str):
        # Use regular expression to extract the first float and the value in brackets
        match = re.match(r'(\d+\.\d+)(?: \((\d+\.\d+)\))?', cell)
        if match:
            return float(match.group(1)), (match.group(2)) if match.group(2) else ''
        else:
            return float('inf'), ''
    else:
        return cell, ''

In [8]:
def highlight_min_max(s):
    """
    Highlight the minimum value in green and the maximum value in red for each column.
    """
    is_min = s == s.min()
    is_max = s == s.max()
    min_max_style = ['background-color: green' if v else '' for v in is_min]
    for i, v in enumerate(is_max):
        if v:
            min_max_style[i] = 'background-color: red'
    return min_max_style

In [9]:
def align_column_names(df):
    if 'case:concept:name' in df.columns:
        df = df.rename(columns={'case:concept:name': 'case_id'})
    elif 'caseid' in df.columns:
        df = df.rename(columns={'caseid': 'case_id'})
    if 'Activity' in df.columns:
        df = df.rename(columns={'Activity': 'activity'})
    elif 'activity_name' in df.columns:
        df = df.rename(columns={'activity_name': 'activity'})
    elif 'task' in df.columns:
        df = df.rename(columns={'task': 'activity'})
    elif 'concept:name' in df.columns:
        df = df.rename(columns={'concept:name': 'activity'})
    if 'Resource' in df.columns:
        df = df.rename(columns={'Resource': 'resource'})
    elif 'user' in df.columns:
        df = df.rename(columns={'user': 'resource'})
    elif 'agent' in df.columns:
        if 'resource' in df.columns:
            df = df.drop(['resource'], axis=1)
        df = df.rename(columns={'agent': 'resource'})
    elif 'org:resource' in df.columns:
        df = df.rename(columns={'org:resource': 'resource'})
    if 'start_timestamp' in df.columns:
        df = df.rename(columns={'start_timestamp': 'start_time'})
    if 'end_timestamp' in df.columns:
        df = df.rename(columns={'end_timestamp': 'end_time'})
    # for SIMOD simulated logs
    if 'start_time' in df.columns:
        df = df.rename(columns={'start_time': 'start_time'})
    if 'end_time' in df.columns:
        df = df.rename(columns={'end_time': 'end_time'})
    if 'start:timestamp' in df.columns:
        df = df.rename(columns={'start:timestamp': 'start_time'})
    if 'time:timestamp' in df.columns:
        df = df.rename(columns={'time:timestamp': 'end_time'})
    return df

In [10]:
def main_(log_paths, name_experiments):
    def perform_evauluation(all_metrics, PATH_SIMULATED_LOG, test_log):
        for i in range(10):
            # print(f"Evaluate simulation {i}")
            path_simulated_file = PATH_SIMULATED_LOG + '/simulated_log_' + str(i) + '.csv'
            # read simulated log and align column names
            simulated_log = pd.read_csv(path_simulated_file)
            simulated_log = align_column_names(simulated_log)
            # print(simulated_log)
            # print("########")
            # print(simulated_log[event_log_ids.activity].unique())
            simulated_log[event_log_ids.start_time] = pd.to_datetime(simulated_log[event_log_ids.start_time], utc=True, format='mixed')
            simulated_log[event_log_ids.end_time] = pd.to_datetime(simulated_log[event_log_ids.end_time], utc=True, format='mixed')

            # Call passing the event logs, and its column ID mappings
            ngd = n_gram_distribution_distance(test_log, event_log_ids, simulated_log, event_log_ids, n=3)
            all_metrics['NGD'].append(ngd)

            # Call passing the event logs, its column ID mappings, timestamp type, and discretize function
            aedd = absolute_event_distribution_distance(
                test_log, event_log_ids,  # First event log and its column id mappings
                simulated_log, event_log_ids,  # Second event log and its column id mappings
                discretize_type=AbsoluteTimestampType.BOTH,  # Type of timestamp distribution (consider start times and/or end times)
                discretize_event=discretize_to_hour  # Function to discretize the absolute seconds of each timestamp (default by hour)
            )
            all_metrics['AEDD'].append(aedd)

            # cadd = case_arrival_distribution_distance(
            #     test_log, event_log_ids,  # First event log and its column id mappings
            #     simulated_log, event_log_ids,  # Second event log and its column id mappings
            #     discretize_event=discretize_to_hour  # Function to discretize each timestamp (default by hour)
            # )
            # all_metrics['CADD'].append(cadd)

            cedd = circadian_event_distribution_distance(
                test_log, event_log_ids,  # First event log and its column id mappings
                simulated_log, event_log_ids,  # Second event log and its column id mappings
                discretize_type=AbsoluteTimestampType.BOTH  # Consider both start/end timestamps of each activity instance
            )
            all_metrics['CEDD'].append(cedd)

            redd = relative_event_distribution_distance(
                test_log, event_log_ids,  # First event log and its column id mappings
                simulated_log, event_log_ids,  # Second event log and its column id mappings
                discretize_type=AbsoluteTimestampType.BOTH,  # Type of timestamp distribution (consider start times and/or end times)
                discretize_event=discretize_to_hour  # Function to discretize the absolute seconds of each timestamp (default by hour)
            )
            all_metrics['REDD'].append(redd)


            ctdd = cycle_time_distribution_distance(
                test_log, event_log_ids,  # First event log and its column id mappings
                simulated_log, event_log_ids,  # Second event log and its column id mappings
                bin_size=pd.Timedelta(hours=1)  # Bins of 1 hour
            )
            all_metrics['CTDD'].append(ctdd)

        return all_metrics
    
    number_evaluations = len(log_paths)

    # Set event log column ID mapping
    event_log_ids = EventLogIDs(  # These values are stored in DEFAULT_CSV_IDS
        case="case_id",
        activity="activity",
        start_time="start_time",
        end_time="end_time",
        resource='resource'
    )

    index_names = name_experiments
    results_df = pd.DataFrame(index=index_names)
    mean_results = pd.DataFrame(index=index_names)

    for experiment in range(number_evaluations):
        # Read and transform time attributes
        test_log = pd.read_csv(log_paths[experiment][0])
        test_log = align_column_names(test_log)
        test_log[event_log_ids.start_time] = pd.to_datetime(test_log[event_log_ids.start_time], utc=True, format='mixed')
        test_log[event_log_ids.end_time] = pd.to_datetime(test_log[event_log_ids.end_time], utc=True, format='mixed')

        PATH_SIMULATED_LOG = log_paths[experiment][1]

        all_metrics = {
            'NGD': [],
            'AEDD': [],
            # 'CADD': [],
            'CEDD': [],
            'REDD': [],
            'CTDD': [],
        }

        all_metrics = perform_evauluation(all_metrics, PATH_SIMULATED_LOG, test_log)

        mean_results.loc[index_names[experiment], 'N-Gram Distribution Distance'] = round(np.mean(all_metrics['NGD']), 3)
        mean_results.loc[index_names[experiment], 'Absolute Event Distribution Distance'] = round(np.mean(all_metrics['AEDD']), 3)
        # mean_results.loc[index_names[experiment], 'Case Arrival Distribution Distance'] = round(np.mean(all_metrics['CADD']), 3)
        mean_results.loc[index_names[experiment], 'Circadian Event Distribution Distance'] = round(np.mean(all_metrics['CEDD']), 3)
        mean_results.loc[index_names[experiment], 'Relative Event Distribution Distance'] = round(np.mean(all_metrics['REDD']), 3)
        mean_results.loc[index_names[experiment], 'Cycle Time Distribution Distance'] = round(np.mean(all_metrics['CTDD']), 3)


    

        results_df.loc[index_names[experiment], 'N-Gram Distribution Distance'] = f"{round(np.mean(all_metrics['NGD']), 3)} ({round(np.std(all_metrics['NGD']), 3)})"
        results_df.loc[index_names[experiment], 'Absolute Event Distribution Distance'] = f"{round(np.mean(all_metrics['AEDD']), 3)} ({round(np.std(all_metrics['AEDD']), 3)})"
        # results_df.loc[index_names[experiment], 'Case Arrival Distribution Distance'] = f"{round(np.mean(all_metrics['CADD']), 3)} ({round(np.std(all_metrics['CADD']), 3)})"
        results_df.loc[index_names[experiment], 'Circadian Event Distribution Distance'] = f"{round(np.mean(all_metrics['CEDD']), 3)} ({round(np.std(all_metrics['CEDD']), 3)})"
        results_df.loc[index_names[experiment], 'Relative Event Distribution Distance'] = f"{round(np.mean(all_metrics['REDD']), 3)} ({round(np.std(all_metrics['REDD']), 3)})"
        results_df.loc[index_names[experiment], 'Cycle Time Distribution Distance'] = f"{round(np.mean(all_metrics['CTDD']), 3)} ({round(np.std(all_metrics['CTDD']), 3)})"

    return mean_results, results_df


### Loan Application

In [11]:
PATH_TEST_LOG_MAS = '../simulated_data/LoanApp.csv/autonomous_smap_smap/test_preprocessed.csv'
PATH_SIMULATED_LOG_MAS = '../simulated_data/LoanApp.csv/autonomous_smap_smap'

PATH_TEST_LOG_LSTM = '../simulated_data/LoanApp.csv/orchestrated_lstm/test_preprocessed.csv'
PATH_SIMULATED_LOG_LSTM = '../simulated_data/LoanApp.csv/orchestrated_lstm'

PATH_TEST_LOG_LSTM_local = '../simulated_data/LoanApp.csv/orchestrated_lstm_local2/test_preprocessed.csv'
PATH_SIMULATED_LOG_LSTM_local = '../simulated_data/LoanApp.csv/orchestrated_lstm_local2'

log_paths = [
    [PATH_TEST_LOG_MAS, PATH_SIMULATED_LOG_MAS],
    [PATH_TEST_LOG_LSTM, PATH_SIMULATED_LOG_LSTM],
    [PATH_TEST_LOG_LSTM_local, PATH_SIMULATED_LOG_LSTM_local],
]

name_experiments = ['AM', 'LSTM', 'LSTM_local']

mean_results, results_df = main_(log_paths, name_experiments)
results_df

,N-Gram Distribution Distance,Absolute Event Distribution Distance,Circadian Event Distribution Distance,Relative Event Distribution Distance,Cycle Time Distribution Distance
AM,0.647 (0.006),13.216 (6.22),0.285 (0.025),7.296 (0.911),3.028 (0.506)
LSTM,0.08 (0.014),3.043 (0.876),0.243 (0.048),2.057 (0.651),2.894 (0.869)
LSTM_local,0.139 (0.016),3.478 (0.999),0.234 (0.032),2.719 (0.614),3.455 (0.978)


## P2P

In [12]:
PATH_TEST_LOG_MAS = '../simulated_data/P2P/autonomous_smap_smap/test_preprocessed.csv'
PATH_SIMULATED_LOG_MAS = '../simulated_data/P2P/autonomous_smap_smap'

PATH_TEST_LOG_LSTM = '../simulated_data/P2P/orchestrated_lstm/test_preprocessed.csv'
PATH_SIMULATED_LOG_LSTM = '../simulated_data/P2P/orchestrated_lstm'

PATH_TEST_LOG_LSTM_local = '../simulated_data/P2P/orchestrated_lstm_local2/test_preprocessed.csv'
PATH_SIMULATED_LOG_LSTM_local = '../simulated_data/P2P/orchestrated_lstm_local2'

log_paths = [
    [PATH_TEST_LOG_MAS, PATH_SIMULATED_LOG_MAS],
    [PATH_TEST_LOG_LSTM, PATH_SIMULATED_LOG_LSTM],
    [PATH_TEST_LOG_LSTM_local, PATH_SIMULATED_LOG_LSTM_local],
]   

name_experiments = ['AM', 'LSTM', 'LSTM_local']

mean_results, results_df = main_(log_paths, name_experiments)
results_df

,N-Gram Distribution Distance,Absolute Event Distribution Distance,Circadian Event Distribution Distance,Relative Event Distribution Distance,Cycle Time Distribution Distance
AM,0.899 (0.013),1295.878 (12.109),1.796 (0.277),827.481 (9.202),664.956 (7.958)
LSTM,0.306 (0.019),1222.752 (5.22),1.422 (0.181),730.29 (5.599),574.817 (6.279)
LSTM_local,0.299 (0.013),1193.583 (6.319),1.134 (0.134),727.042 (6.706),559.332 (6.263)


### Confidential_1000

In [14]:
PATH_TEST_LOG_MAS = '../simulated_data/Confidential_1000/autonomous_smap_smap/test_preprocessed.csv'
PATH_SIMULATED_LOG_MAS = '../simulated_data/Confidential_1000/autonomous_smap_smap'

PATH_TEST_LOG_LSTM = '../simulated_data/Confidential_1000/autonomous_lstm/test_preprocessed.csv'
PATH_SIMULATED_LOG_LSTM = '../simulated_data/Confidential_1000/autonomous_lstm'

PATH_TEST_LOG_LSTM_local = '../simulated_data/Confidential_1000/autonomous_lstm_local2/test_preprocessed.csv'
PATH_SIMULATED_LOG_LSTM_local = '../simulated_data/Confidential_1000/autonomous_lstm_local2'

log_paths = [
    [PATH_TEST_LOG_MAS, PATH_SIMULATED_LOG_MAS],
    [PATH_TEST_LOG_LSTM, PATH_SIMULATED_LOG_LSTM],
    [PATH_TEST_LOG_LSTM_local, PATH_SIMULATED_LOG_LSTM_local],
]

name_experiments = ['AM', 'LSTM', 'LSTM_local']

mean_results, results_df = main_(log_paths, name_experiments)
results_df

,N-Gram Distribution Distance,Absolute Event Distribution Distance,Circadian Event Distribution Distance,Relative Event Distribution Distance,Cycle Time Distribution Distance
AM,0.804 (0.004),152.964 (8.996),6.032 (1.136),9.485 (0.517),7.563 (0.637)
LSTM,0.241 (0.01),118.163 (9.407),2.509 (0.284),25.427 (8.917),41.335 (14.375)
LSTM_local,0.295 (0.006),116.068 (14.531),3.018 (0.267),26.378 (11.589),42.484 (17.143)


In [15]:
# PATH_TEST_LOG_MAS = '../simulated_data/Confidential_2000/autonomous_smap_smap/test_preprocessed.csv'
# PATH_SIMULATED_LOG_MAS = '../simulated_data/Confidential_2000/autonomous_smap_smap'

PATH_TEST_LOG_LSTM = '../simulated_data/Confidential_2000/autonomous_lstm/test_preprocessed.csv'
PATH_SIMULATED_LOG_LSTM = '../simulated_data/Confidential_2000/autonomous_lstm'

PATH_TEST_LOG_LSTM_local = '../simulated_data/Confidential_2000/autonomous_lstm_local2/test_preprocessed.csv'
PATH_SIMULATED_LOG_LSTM_local = '../simulated_data/Confidential_2000/autonomous_lstm_local2'

log_paths = [
    # [PATH_TEST_LOG_MAS, PATH_SIMULATED_LOG_MAS],
    [PATH_TEST_LOG_LSTM, PATH_SIMULATED_LOG_LSTM],
    [PATH_TEST_LOG_LSTM_local, PATH_SIMULATED_LOG_LSTM_local],
]

name_experiments = ['LSTM', 'LSTM_local']

mean_results, results_df = main_(log_paths, name_experiments)
results_df

,N-Gram Distribution Distance,Absolute Event Distribution Distance,Circadian Event Distribution Distance,Relative Event Distribution Distance,Cycle Time Distribution Distance
LSTM,0.289 (0.011),195.661 (5.46),2.775 (1.006),14.751 (5.681),26.185 (8.915)
LSTM_local,0.279 (0.007),202.001 (17.626),2.909 (1.046),46.142 (18.69),78.79 (26.707)


In [16]:
PATH_TEST_LOG_MAS = '../simulated_data/BPIC_2012_W/autonomous_smap_smap/test_preprocessed.csv'
PATH_SIMULATED_LOG_MAS = '../simulated_data/BPIC_2012_W/autonomous_smap_smap'

PATH_TEST_LOG_LSTM = '../simulated_data/BPIC_2012_W/orchestrated_lstm/test_preprocessed.csv'
PATH_SIMULATED_LOG_LSTM = '../simulated_data/BPIC_2012_W/orchestrated_lstm'

PATH_TEST_LOG_LSTM_local = '../simulated_data/BPIC_2012_W/orchestrated_lstm_local2/test_preprocessed.csv'
PATH_SIMULATED_LOG_LSTM_local = '../simulated_data/BPIC_2012_W/orchestrated_lstm_local2'

log_paths = [
    [PATH_TEST_LOG_MAS, PATH_SIMULATED_LOG_MAS],
    [PATH_TEST_LOG_LSTM, PATH_SIMULATED_LOG_LSTM],
    [PATH_TEST_LOG_LSTM_local, PATH_SIMULATED_LOG_LSTM_local],
]   

name_experiments = ['AM', 'LSTM', 'LSTM_local']

mean_results, results_df = main_(log_paths, name_experiments)
results_df

,N-Gram Distribution Distance,Absolute Event Distribution Distance,Circadian Event Distribution Distance,Relative Event Distribution Distance,Cycle Time Distribution Distance
AM,0.302 (0.007),80.95 (7.652),1.966 (0.071),79.483 (4.223),115.238 (2.889)
LSTM,0.166 (0.007),77.206 (8.512),1.831 (0.094),41.02 (6.838),72.984 (4.12)
LSTM_local,0.277 (0.019),206.02 (34.72),1.798 (0.059),160.213 (35.587),114.262 (11.522)


In [19]:
PATH_TEST_LOG_MAS = '../simulated_data/BPIC_2017_W/autonomous_smap_smap/test_preprocessed.csv'
PATH_SIMULATED_LOG_MAS = '../simulated_data/BPIC_2017_W/autonomous_smap_smap'

PATH_TEST_LOG_LSTM = '../simulated_data/BPIC_2017_W/orchestrated_lstm/test_preprocessed.csv'
PATH_SIMULATED_LOG_LSTM = '../simulated_data/BPIC_2017_W/orchestrated_lstm'

PATH_TEST_LOG_LSTM_local = '../simulated_data/BPIC_2017_W/orchestrated_lstm_local2/test_preprocessed.csv'
PATH_SIMULATED_LOG_LSTM_local = '../simulated_data/BPIC_2017_W/orchestrated_lstm_local2'

log_paths = [
    [PATH_TEST_LOG_MAS, PATH_SIMULATED_LOG_MAS],
    [PATH_TEST_LOG_LSTM, PATH_SIMULATED_LOG_LSTM],
    [PATH_TEST_LOG_LSTM_local, PATH_SIMULATED_LOG_LSTM_local],
]   

name_experiments = ['AM', 'LSTM', 'LSTM_local']

mean_results, results_df = main_(log_paths, name_experiments)
results_df

FileNotFoundError: [Errno 2] No such file or directory: '../simulated_data/BPIC_2017_W/orchestrated_lstm/test_preprocessed.csv'

In [18]:
PATH_TEST_LOG_MAS = '../simulated_data/ConsultaDataMining/autonomous_smap_smap/test_preprocessed.csv'
PATH_SIMULATED_LOG_MAS = '../simulated_data/ConsultaDataMining/autonomous_smap_smap'

PATH_TEST_LOG_LSTM = '../simulated_data/ConsultaDataMining/orchestrated_lstm/test_preprocessed.csv'
PATH_SIMULATED_LOG_LSTM = '../simulated_data/ConsultaDataMining/orchestrated_lstm'

PATH_TEST_LOG_LSTM_local = '../simulated_data/ConsultaDataMining/orchestrated_lstm_local2/test_preprocessed.csv'
PATH_SIMULATED_LOG_LSTM_local = '../simulated_data/ConsultaDataMining/orchestrated_lstm_local2'

log_paths = [
    [PATH_TEST_LOG_MAS, PATH_SIMULATED_LOG_MAS],
    [PATH_TEST_LOG_LSTM, PATH_SIMULATED_LOG_LSTM],
    [PATH_TEST_LOG_LSTM_local, PATH_SIMULATED_LOG_LSTM_local],
]   

name_experiments = ['AM', 'LSTM', 'LSTM_local']

mean_results, results_df = main_(log_paths, name_experiments)
results_df

,N-Gram Distribution Distance,Absolute Event Distribution Distance,Circadian Event Distribution Distance,Relative Event Distribution Distance,Cycle Time Distribution Distance
AM,0.76 (0.014),328.956 (2.196),5.085 (0.219),25.444 (1.324),74.638 (1.198)
LSTM,0.404 (0.015),278.781 (1.978),6.048 (0.28),24.514 (0.981),71.675 (2.101)
LSTM_local,0.471 (0.025),315.989 (3.832),6.942 (0.29),23.014 (1.584),67.894 (2.221)


In [20]:
PATH_TEST_LOG_MAS = '../simulated_data/cvs_pharmacy/autonomous_smap_smap/test_preprocessed.csv'
PATH_SIMULATED_LOG_MAS = '../simulated_data/cvs_pharmacy/autonomous_smap_smap'

PATH_TEST_LOG_LSTM = '../simulated_data/cvs_pharmacy/orchestrated_lstm/test_preprocessed.csv'
PATH_SIMULATED_LOG_LSTM = '../simulated_data/cvs_pharmacy/orchestrated_lstm'

PATH_TEST_LOG_LSTM_local = '../simulated_data/cvs_pharmacy/orchestrated_lstm_local2/test_preprocessed.csv'
PATH_SIMULATED_LOG_LSTM_local = '../simulated_data/cvs_pharmacy/orchestrated_lstm_local2'

log_paths = [
    [PATH_TEST_LOG_MAS, PATH_SIMULATED_LOG_MAS],
    [PATH_TEST_LOG_LSTM, PATH_SIMULATED_LOG_LSTM],
    [PATH_TEST_LOG_LSTM_local, PATH_SIMULATED_LOG_LSTM_local],
]   

name_experiments = ['AM', 'LSTM', 'LSTM_local']

mean_results, results_df = main_(log_paths, name_experiments)
results_df

,N-Gram Distribution Distance,Absolute Event Distribution Distance,Circadian Event Distribution Distance,Relative Event Distribution Distance,Cycle Time Distribution Distance
AM,0.852 (0.002),117.977 (1.384),7.571 (0.026),110.996 (1.241),196.818 (2.618)
LSTM,0.101 (0.006),100.745 (1.361),7.588 (0.037),94.482 (1.5),119.236 (1.991)
LSTM_local,0.113 (0.004),98.176 (0.972),7.564 (0.028),90.465 (0.981),113.823 (2.225)


In [21]:
PATH_TEST_LOG_MAS = '../simulated_data/Production/autonomous_smap_smap/test_preprocessed.csv'
PATH_SIMULATED_LOG_MAS = '../simulated_data/Production/autonomous_smap_smap'

PATH_TEST_LOG_LSTM = '../simulated_data/Production/orchestrated_lstm/test_preprocessed.csv'
PATH_SIMULATED_LOG_LSTM = '../simulated_data/Production/orchestrated_lstm'

PATH_TEST_LOG_LSTM_local = '../simulated_data/Production/orchestrated_lstm_local2/test_preprocessed.csv'
PATH_SIMULATED_LOG_LSTM_local = '../simulated_data/Production/orchestrated_lstm_local2'

log_paths = [
    [PATH_TEST_LOG_MAS, PATH_SIMULATED_LOG_MAS],
    [PATH_TEST_LOG_LSTM, PATH_SIMULATED_LOG_LSTM],
    [PATH_TEST_LOG_LSTM_local, PATH_SIMULATED_LOG_LSTM_local],
]   

name_experiments = ['AM', 'LSTM', 'LSTM_local']

mean_results, results_df = main_(log_paths, name_experiments)
results_df

,N-Gram Distribution Distance,Absolute Event Distribution Distance,Circadian Event Distribution Distance,Relative Event Distribution Distance,Cycle Time Distribution Distance
AM,0.733 (0.022),92.835 (7.505),5.989 (0.419),47.648 (5.541),64.486 (4.077)
LSTM,0.57 (0.019),100.853 (11.051),5.483 (0.738),34.618 (9.813),24.8 (9.469)
LSTM_local,0.826 (0.022),81.037 (11.971),5.961 (0.783),26.952 (5.655),20.032 (7.286)
